In [73]:
import pandas as pd

In [74]:
import numpy as np

In [75]:
from dynamic_portfolio.utils import *
from dynamic_portfolio.preprocess import *

# Loading data and creating clean dataframe

In [ ]:
df = features_creation('META')

In [ ]:
pd.set_option('display.max_columns', None)

# Baseline

In [ ]:
backtest_mask = final_df['date'] > '2020-01-02'

In [ ]:
backtest_df = final_df[backtest_mask]

In [ ]:
model_mask = final_df['date'] <= '2020-01-02'

In [ ]:
model_df = final_df[model_mask]

In [ ]:
model_df[['return']].iloc[-1]

In [ ]:
backtest_df[['return']].iloc[0]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(backtest_df[['return']].iloc[0],model_df[['return']].iloc[-1])**0.5

In [ ]:
def baseline_model(df):
    df = y_train
    y_pred = y_train[-1]
    return y_pred

In [ ]:
def rmse():
    rmse = (np.mean((y_test[0]-y_pred)**2))**0.5
    return rmse

In [ ]:
X = model_df.drop(columns=['date'])

In [ ]:
y = model_df['return']

In [ ]:
model = model.fit(X,y)

In [ ]:
backtest_df = backtest_df.drop(columns=['date'])

In [ ]:
y_true = backtest_df['return']

In [ ]:
y_true

In [ ]:
backtest_df.iloc[[3]]

In [ ]:
backtest_df

In [ ]:
y_pred = []
backtest_df.reset_index(drop=True, inplace=True)
for i in range(len(backtest_df)):
    pred = model.predict(backtest_df.iloc[[i]])
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
y_pred[:,0].shape

In [ ]:
y_true.shape

In [ ]:
rmse = (np.mean((y_true-y_pred[:,0])**2))**0.5

In [ ]:
rmse

# Cross vals

We use sklearn's time series split to break up the data in different folds
We use a sklearn example to run the first tests.

Metrics used : 
 - rmse
 - mae
 - R2

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
ts_cv = TimeSeriesSplit(
    n_splits=20,
    gap=0,
    max_train_size=252,
    test_size=45,
)

In [76]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [5]:
def baseline_model():
    

SyntaxError: incomplete input (671652115.py, line 2)

In [6]:
model = HistGradientBoostingRegressor()

In [ ]:
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.6f} +/- {mae.std():.6f}\n"
        f"Root Mean Squared Error: {rmse.mean():.6f} +/- {rmse.std():.6f}"
    )


In [ ]:
import seaborn as sns

In [ ]:
evaluate(model, X, y, ts_cv)

In [ ]:
scores = cross_val_score(model, X, y, cv = ts_cv)

In [ ]:
scores = np.mean(scores)
scores

# PCA Analysis

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 0.95)
X_pca = pca.fit_transform(aapl) 
print(pca.n_components_) 

In [ ]:
pca.n_components

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(np.unique(pd.DataFrame(pca.components_, columns = aapl.columns).columns))


In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
n_pcs= pca.n_components_ 
# get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = aapl.columns
# get the most important feature names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

In [ ]:
n_pcs

In [ ]:
len(np.unique(most_important_names))

In [ ]:
np.unique(most_important_names)

In [64]:
from dynamic_portfolio.cross_validate import *

In [65]:
from sklearn.metrics import mean_squared_error

In [77]:
fold_length = 125 #
fold_stride = 60 #1 trimestre
train_test_ratio = 0.7
input_length = 10
horizon = 1
output_length = 1
#stride = 1

In [78]:
def get_folds(
    df: pd.DataFrame,
    fold_length: int,
    fold_stride: int):
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold

    Returns a list of folds, each as a DataFrame
    '''

    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds

In [98]:
df = ready_to_train_df('AAPL')

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [99]:
df

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,...,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,-0.105370,0.0,0.0,-0.375034,0.0,1.580927,1.903384,-1.342731,-1.252546,-0.934743,...,0.418402,-0.689312,-0.581609,0.752685,0.879385,-1.530922,0.00,0.673519,-2.405975,-0.036933
1,-0.545803,0.0,0.0,-0.375034,0.0,1.643851,1.923358,-1.293905,-1.189135,-0.934743,...,-0.104653,0.693065,0.125781,0.528113,2.282965,1.285421,0.00,0.673519,-2.405975,-0.036933
2,-0.797475,0.0,0.0,-0.375034,0.0,1.697786,1.923358,-1.220667,-1.185778,-0.934743,...,0.668578,0.585609,-0.018560,0.718659,0.094866,-0.737712,0.00,0.673519,-2.405975,-0.036933
3,-0.445201,0.0,0.0,-0.375034,0.0,1.742732,1.969965,-1.245080,-1.206294,-0.934743,...,-0.170887,0.481934,0.316109,-0.205489,-0.724499,0.004588,0.00,0.673519,-2.405975,-0.036933
4,-0.604064,0.0,0.0,-0.375034,0.0,1.769699,1.963307,-1.196254,-1.213754,-0.934743,...,-0.571964,0.285290,-0.065675,0.440779,-0.281032,2.559282,0.00,0.673519,-2.405975,-0.036933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4342,-0.354323,0.0,0.0,1.483221,0.0,-0.468592,0.438594,-1.440382,0.352482,1.133826,...,-0.303289,0.177038,-0.018560,0.173144,-2.129337,0.209828,0.00,-0.118737,0.073284,0.983225
4343,-0.060090,0.0,0.0,1.483221,0.0,-0.459603,0.438594,-1.428176,0.377847,1.133826,...,0.274074,0.176405,-0.018560,0.167862,0.360989,-1.553625,0.00,-0.118737,0.073284,0.983225
4344,-0.334273,0.0,0.0,1.483221,0.0,-0.441625,0.471885,-1.464795,0.368895,0.129175,...,-0.795470,0.354368,0.432028,-0.472254,-0.154089,0.500617,-0.05,-0.107414,-0.670808,1.383396
4345,-0.541103,0.0,0.0,1.483221,0.0,-0.477581,0.445253,-1.464795,0.380085,0.129175,...,0.739964,-0.712206,-0.372018,0.004111,0.146726,-0.572649,0.00,-0.107414,-0.670808,0.925382


In [103]:
len(train_test_split(df, 0.7, 15,1)[0])+ len(train_test_split(df, 0.7, 15,1)[1])

4362

In [79]:
def train_test_split(fold: pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int,
                     horizon: int) :
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    '''

    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]

    # TEST SET
    # ======================
    first_test_idx = last_train_idx - input_length
    fold_test = fold.iloc[first_test_idx:, :]

    return (fold_train, fold_test)



In [80]:
def cross_validate_ml(df) :
    '''
    get_folds() create many FOLDS, train_test_split() create a split on ONE FOLDS.
    The goal of this function is to make splits and sequences on each FOLDS.
    Then, apply a model.
    '''
    folds = get_folds(df, fold_length, fold_stride) # 1 - Creating FOLDS
    rmses_model = []
    rmses_baseline = []
    
    for fold_id, fold in enumerate(folds):

        # 2 - CHRONOLOGICAL TRAIN TEST SPLIT of the current FOLD

        (fold_train, fold_test) = train_test_split(fold = fold,
                                                train_test_ratio=train_test_ratio,
                                                input_length = input_length,
                                                horizon=horizon)

        # 3 - Scanninng fold_train and fold_test for SEQUENCES

        X_train, y_train = fold_train, fold_train['return']

        X_test, y_test = fold_test, fold_test['return']

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        rmse_model = (mean_squared_error(y_test, model.predict(X_test)))**0.5
        rmses_model.append(rmse_model)
        
        #print(type(y_train))
        #rmse_baseline = mean_squared_error(y_test.iloc[[0]], y_train.iloc[[-1]])**0.5
        #rmses_baseline.append(rmse_baseline)
        
    
        
        
        
    return np.mean(rmses_model)
        
            

In [42]:
model, baseline = cross_validate_ml(df)[0], cross_validate_ml(df)[1]

In [81]:
cross_validate_ml(df)

0.008603253748945635

In [82]:
df

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,...,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,-0.998390,0.0,0.0,0.448416,0.0,0.801740,-0.569891,1.133770,0.171681,-0.317363,...,-0.419313,1.187464,0.743017,0.301611,-0.068194,1.718868,0.0,1.940652,-0.456106,0.875470
1,-1.032198,0.0,0.0,0.448416,0.0,0.603962,-0.609053,1.030990,0.148834,-0.317363,...,0.723640,-1.392291,-1.117542,-0.365681,-0.081601,-0.770408,0.0,1.940652,-0.456106,0.875470
2,-1.024241,0.0,0.0,0.448416,0.0,0.693861,-0.569891,1.056685,0.177181,-0.317363,...,-0.033843,0.666983,1.147240,0.075767,0.141698,-0.637953,0.0,1.940652,-0.456106,0.875470
3,-1.008716,0.0,0.0,0.448416,0.0,0.891639,-0.504622,1.133770,0.160258,-0.317363,...,-0.915490,1.433922,1.812472,0.255088,-0.055741,0.199758,0.0,1.940652,-0.456106,0.875470
4,-1.006878,0.0,0.0,0.448416,0.0,0.963558,-0.491568,1.172313,0.251221,-0.317363,...,0.174107,0.504010,0.315338,0.116275,0.414262,0.064280,0.0,1.940652,-0.456106,0.875470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,0.306056,0.0,0.0,0.702970,0.0,-2.452599,-0.961507,-0.806206,-1.310391,-0.312340,...,-1.523843,3.797413,2.988333,0.902844,0.775337,0.584562,0.0,0.740619,5.821427,-8.921692
2430,1.431119,0.0,0.0,0.702970,0.0,-2.506539,-0.961507,-0.844749,-1.366238,-0.312340,...,0.546709,-1.134041,-0.018786,-0.434260,-0.440184,-0.755108,0.0,0.740619,5.821427,-8.921692
2431,0.938401,0.0,0.0,0.702970,0.0,-2.578458,-0.987615,-0.870444,-1.331968,-0.312340,...,1.163206,-1.565516,-1.750158,-0.307262,0.308312,-0.711269,0.0,0.740619,5.821427,-8.921692
2432,2.207120,0.0,0.0,0.702970,0.0,-2.740276,-1.026776,-0.947529,-1.308276,-0.312340,...,1.279844,-3.697007,-2.875550,-0.918836,0.214426,-0.852736,0.0,0.740619,5.821427,-8.921692


In [ ]:
improvement = baseline/model
improvement

In [ ]:
baseline

In [43]:
model

0.00373410786557851

In [84]:
tickers = return_tickers()

In [93]:
tickers

['AAPL',
 'MSFT',
 'GOOG',
 'AMZN',
 'TSLA',
 'UNH',
 'XOM',
 'JNJ',
 'WMT',
 'NVDA',
 'JPM',
 'V',
 'CVX',
 'PG',
 'LLY',
 'MA',
 'HD',
 'META',
 'BAC',
 'ABBV',
 'PFE',
 'KO',
 'MRK',
 'PEP',
 'COST',
 'ORCL',
 'AVGO',
 'TMO',
 'MCD',
 'CSCO',
 'ACN',
 'DHR',
 'TMUS',
 'ABT',
 'WFC',
 'DIS',
 'LIN',
 'NEE',
 'BMY',
 'NKE',
 'VZ',
 'TXN',
 'UPS',
 'COP',
 'ADBE',
 'CMCSA',
 'CRM',
 'PM',
 'MS',
 'AMGN',
 'SCHW',
 'HON',
 'RTX',
 'QCOM',
 'T',
 'IBM',
 'DE',
 'CVS',
 'LOW',
 'GS',
 'UNP',
 'NFLX',
 'LMT',
 'CAT',
 'AMD',
 'INTC',
 'ELV',
 'SPGI',
 'AXP',
 'SBUX',
 'INTU',
 'BLK',
 'ADP',
 'GILD',
 'PLD',
 'MDT',
 'BA',
 'AMT',
 'CI',
 'GE',
 'TJX',
 'ISRG',
 'C',
 'AMAT',
 'PYPL',
 'MDLZ',
 'CB',
 'SYK',
 'ADI',
 'MMC',
 'EOG',
 'NOW',
 'VRTX',
 'MO',
 'NOC',
 'EL',
 'REGN',
 'PGR',
 'BKNG',
 'DUK',
 'TGT',
 'SLB',
 'SO',
 'MMM',
 'ITW',
 'ZTS',
 'GD',
 'APD',
 'HUM',
 'MRNA',
 'BDX',
 'CSX',
 'WM',
 'PNC',
 'HCA',
 'ETN',
 'USB',
 'FISV',
 'SHW',
 'OXY',
 'CL',
 'MU',
 'CME',
 'AON',


In [92]:
tickers.remove('DLRKEYS')

In [85]:
from sklearn.ensemble import RandomForestRegressor

In [96]:
apple = ready_to_train_df('AAPL')

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [94]:
rmses = []


for ticker in tickers:
    df = ready_to_train_df(ticker)
    rmse = cross_validate_ml(df)
    rmses.append(rmse)


/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

...

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [ ]:
np.mean(rmses)

In [3]:
import pandas as pd
import numpy as np
import dynamic_portfolio.preprocess as prep
import dynamic_portfolio.utils as utils
import dynamic_portfolio.cross_validate as cv
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import warnings

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

warnings.filterwarnings(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
prep.ready_to_train_df('AAPL')

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,cpi,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,volume_momentum_5days,volume_momentum_10days,volume_momentum_20days,price/eps,momentum_5days/eps,momentum_10days/eps,momentum_20days/eps,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-18,-0.105370,0.0,0.0,-0.375034,0.0,1.580927,1.903384,-1.342731,-1.252546,-0.934743,2.606178,-0.555556,0.2,-0.610978,-1.052896,-1.238462,-1.597627,-1.491827,1.864062,0.111838,1.842193,2.695920,2.923183,1.994286,-0.478714,1.685335,1.643434,-0.719437,-1.442745,-1.240978,-1.190488,-1.117295,-1.379262,-0.812372,-0.848085,-0.994603,-0.817374,0.046891,-0.565027,-0.625177,0.418402,-0.689312,-0.581609,0.752685,0.879385,-1.530922,0.00,0.673519,-2.405975,-0.036933
2001-01-19,-0.545803,0.0,0.0,-0.375034,0.0,1.643851,1.923358,-1.293905,-1.189135,-0.934743,2.600313,-0.555556,0.2,-0.610978,-1.052896,-1.238462,-1.597627,-1.492268,1.928136,0.043339,0.944023,3.980977,3.129773,2.609049,2.468842,1.849609,2.622334,4.175244,4.443153,4.337933,-1.559069,-1.634087,-1.748852,-0.671165,-0.801449,-0.941540,-0.830490,-0.764047,-0.609493,-0.904996,-0.104653,0.693065,0.125781,0.528113,2.282965,1.285421,0.00,0.673519,-2.405975,-0.036933
2001-01-22,-0.797475,0.0,0.0,-0.375034,0.0,1.697786,1.923358,-1.220667,-1.185778,-0.934743,2.647238,-0.555556,0.2,-0.610978,-1.052896,-1.238462,-1.597627,-1.487638,1.890759,-0.012821,1.835024,3.398468,3.136038,2.585941,1.603004,1.866289,3.228468,1.256646,1.306832,1.145600,-0.498298,-0.798034,-1.045310,-0.677080,-0.797209,-0.917831,-0.826442,-0.525836,-0.614008,-1.078259,0.668578,0.585609,-0.018560,0.718659,0.094866,-0.737712,0.00,0.673519,-2.405975,-0.036933
2001-01-23,-0.445201,0.0,0.0,-0.375034,0.0,1.742732,1.969965,-1.245080,-1.206294,-0.934743,2.629641,-0.555556,0.2,-0.610978,-1.052896,-1.238462,-1.597627,-1.488520,1.890759,0.064935,3.604759,2.843761,2.909040,2.630055,2.241785,2.254032,2.830601,-1.799599,-1.386286,-1.271107,1.070367,0.376331,-0.162741,-0.656114,-0.785039,-0.920540,-0.846683,-0.701579,-0.718961,-0.964529,-0.170887,0.481934,0.316109,-0.205489,-0.724499,0.004588,0.00,0.673519,-2.405975,-0.036933
2001-01-24,-0.604064,0.0,0.0,-0.375034,0.0,1.769699,1.963307,-1.196254,-1.213754,-0.934743,2.676566,-0.555556,0.2,-0.610978,-1.052896,-1.238462,-1.597627,-1.492047,2.018907,0.000000,1.517272,2.830914,3.095191,2.664827,3.611615,3.002549,3.698629,1.323002,1.923920,2.003887,-0.356491,-0.869106,-1.189121,-0.530819,-0.727494,-0.890353,-0.846683,-1.078450,-0.921569,-1.212655,-0.571964,0.285290,-0.065675,0.440779,-0.281032,2.559282,0.00,0.673519,-2.405975,-0.036933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-28,-0.354323,0.0,0.0,1.483221,0.0,-0.468592,0.438594,-1.440382,0.352482,1.133826,0.248183,-0.629630,0.2,2.113877,0.891249,1.274757,1.789831,0.675707,0.569240,0.007276,-0.764020,-0.786199,-1.075900,-1.247985,-0.390281,-0.688282,-0.465178,0.049690,0.090921,-0.025803,-0.043466,0.123391,-0.271621,0.186912,0.332670,0.128462,0.157298,-0.046218,-0.055448,-0.072177,-0.303289,0.177038,-0.018560,0.173144,-2.129337,0.209828,0.00,-0.118737,0.073284,0.983225
2018-06-29,-0.060090,0.0,0.0,1.483221,0.0,-0.459603,0.438594,-1.428176,0.377847,1.133826,0.248183,-0.629630,0.2,2.113877,0.891249,1.274757,1.789831,0.685343,0.501606,-0.002102,-0.238512,-0.733840,-1.022150,-1.231453,-0.135148,-0.593513,-0.359285,0.350930,0.45954

In [4]:
prep.ready_to_test_df('AAPL')

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,cpi,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,volume_momentum_5days,volume_momentum_10days,volume_momentum_20days,price/eps,momentum_5days/eps,momentum_10days/eps,momentum_20days/eps,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-07-09,-0.223353,0.0,0.0,1.483221,0.0,-0.450614,0.471885,-1.477001,0.370387,0.129175,0.248183,-0.703704,0.0,1.949215,0.891679,1.252746,1.897966,0.696631,0.452661,0.013885,-1.061426,-0.547813,-0.856703,-1.178957,0.125635,-0.007528,-0.571539,0.544191,0.569736,0.681885,-0.205325,0.340758,0.623692,-0.374948,-0.150586,0.012531,0.175702,-0.044201,-0.052829,-0.072610,0.187388,0.719152,0.340486,0.004111,0.063171,0.087778,0.0,-0.107414,-0.670808,0.925382
2018-07-10,-0.432426,0.0,0.0,1.483221,0.0,-0.441625,0.485201,-1.489208,0.377101,0.129175,0.248183,-0.703704,0.0,1.949215,0.891679,1.252746,1.897966,0.691913,0.455331,-0.001207,-1.138358,-0.508697,-0.807630,-1.137700,0.442998,0.228232,-0.342728,0.389206,0.494027,0.631648,-0.671704,-0.153611,0.180242,-0.411425,-0.198009,-0.010221,0.174869,-0.042960,-0.051922,-0.071680,-0.188871,0.175152,0.158169,-0.176579,0.079728,0.066950,0.0,-0.107414,-0.670808,0.925382
2018-07-11,-0.274022,0.0,0.0,1.483221,0.0,-0.459603,0.478543,-1.501414,0.241328,0.129175,0.248183,-0.703704,0.0,1.949215,0.891679,1.252746,1.897966,0.662214,0.509616,-0.012976,-0.842970,-0.514648,-0.960345,-1.138799,0.197062,0.420026,-0.330334,-0.226688,-0.258237,-0.044548,-0.083420,0.268023,0.886953,-0.433355,-0.298183,-0.018033,0.165920,-0.043921,-0.051184,-0.071630,-0.992113,-0.357511,-0.106242,-0.183032,-2.045247,1.272267,0.0,-0.107414,-0.670808,0.925382
2018-07-12,-0.317306,0.0,0.0,1.483221,0.0,-0.459603,0.491859,-1.525827,0.234241,0.129175,0.248183,-0.703704,0.0,1.949215,0.891679,1.252746,1.897966,0.673260,0.517624,0.016766,-0.867164,-0.653312,-0.874234,-1.107967,0.285204,0.063764,-0.514186,-0.844690,-0.680158,-0.528501,-0.432121,-0.326314,0.326049,-0.373682,-0.334738,-0.011702,0.177332,-0.043577,-0.052555,-0.072377,0.321460,-0.002816,0.157484,-0.384038,-0.131773,0.190411,0.0,-0.107414,-0.670808,0.925382
2018-07-13,-0.619695,0.0,0.0,1.483221,0.0,-0.477581,0.485201,-1.538033,0.262216,0.129175,0.248183,-0.703704,0.0,1.949215,0.891679,1.252746,1.897966,0.660538,0.512285,0.001570,-1.182286,-0.548572,-0.794796,-1.046641,0.458282,0.319639,-0.265459,0.522345,0.595934,0.742877,-0.250252,-0.317898,0.439909,-0.356099,-0.380889,-0.023563,0.178419,-0.042900,-0.051570,-0.071366,-0.449171,-0.360000,-0.105905,-0.205489,0.419258,-0.119176,0.0,-0.107414,-0.670808,0.925382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-16,2.211691,0.0,0.0,3.114094,0.0,0.277505,1.657033,-2.661021,0.806426,1.398670,1.374390,-0.740741,4.2,2.741831,2.054944,3.056036,3.000000,1.835211,1.538354,-0.008331,-0.539033,2.018152,1.588741,1.277056,1.472629,-0.398754,0.346495,-0.215724,0.606065,0.387252,-0.262018,-0.151381,-0.221487,4.010730,4.490035,4.583137,0.675563,-0.041910,-0.053981,-0.070105,-0.261585,-1.744089,-0.122494,1.015342,-0.613775,-0.215991,0.0,-0.385819,-0.352459,-0.924672
2022-11-17,3.097340,0.0,0.0,3.114094,0.0,0.367396,1.710298,-2.636608,0.659836,1.398670,1.374390,-0.740741,4.2,2.741831,2.054944,3.056036,3.000000,1.806768,1.574841,0.012971,0.429132,2.113816,1.558560,1.283421,1.224375,-0.269460,0.169874,-1.079904,-0.359339,-0.474532,0.747452,0.938015,